In [1]:
import pandas
import numpy as np

Retrieving business, training and testing reviews dataframe from pickle

In [2]:
business_df = pandas.read_pickle('business.pkl')
review_train_df = pandas.read_pickle('reviews_train.pkl')
review_test_df = pandas.read_pickle('reviews_test.pkl')

Calculating mean rating of each user and restaurant in a dataframe

In [3]:
avg_user_rating_df = review_train_df.groupby(["user_id"]).mean() #average of all the ratings by a particular user
avg_business_rating_df = review_train_df.groupby(["business_id"]).mean() #average of all the ratings for a particular business

Hash user ids to matrix indices

In [4]:
user_id_hash = {}
user_ids = review_train_df["user_id"].unique()
for i,v in enumerate(user_ids):
    user_id_hash[v] = i

Hash business ids to matrix indices

In [5]:
business_id_hash = {}
business_ids = review_train_df["business_id"].unique()
for i,v in enumerate(business_ids):
    business_id_hash[v] = i

Create utility matrix

In [6]:
R = len(review_train_df["business_id"].unique())
C = len(review_train_df["user_id"].unique())
U = np.empty((R,C))
U[:] = np.nan

In [7]:
def populate_U(business_id,user_id,rating):
    U[business_id_hash[business_id],user_id_hash[user_id]] = rating

Populate utility matrix

In [8]:
review_train_df.apply(lambda x: populate_U(x.business_id,x.user_id,x.stars),axis = 1)

3562529    None
2754594    None
1897001    None
1907883    None
3857585    None
4880337    None
4913059    None
1923776    None
5698276    None
4935296    None
5467597    None
1898143    None
4261037    None
2890068    None
3968651    None
5527622    None
1060883    None
1827374    None
5961628    None
3260708    None
825253     None
3972541    None
4286269    None
1675161    None
3191235    None
2604962    None
5202627    None
3410603    None
4414310    None
3928981    None
           ... 
3283934    None
1898212    None
3592181    None
2902069    None
3446399    None
4205765    None
2848846    None
4148538    None
3548395    None
1445398    None
4227317    None
1134243    None
3705608    None
2823989    None
2658182    None
4811842    None
4672945    None
1702837    None
5224278    None
5641665    None
2271982    None
1326890    None
2995707    None
1966357    None
2061353    None
3516266    None
4590444    None
2215095    None
2367196    None
2091795    None
Length: 409051, dtype: o

Calculating row mean for every row (ratings for every item) excluding nan values which represent no rating

In [9]:
row_means = np.nanmean(U, axis=1)

Centering rating for every business around its mean rating

In [ ]:
row_means_col_vec = row_means.reshape((len(U), 1))
centered_U = U - row_means_col_vec
centered_U = np.nan_to_num(centered_U)

Dumping centered utility matrix to file

In [18]:
centered_U.dump("centered_U.dat")

OverflowError: cannot serialize a string larger than 4GiB

array([[ 1.09090909,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.51515152,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.22099448, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])